In [153]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.dates import YearLocator, MonthLocator
import scipy

In [154]:
raw = pd.read_csv('./data/tr_eikon_eod_data.csv', 
                  index_col=0, parse_dates=True)
raw.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2216 entries, 2010-01-01 to 2018-06-29
Data columns (total 12 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL.O  2138 non-null   float64
 1   MSFT.O  2138 non-null   float64
 2   INTC.O  2138 non-null   float64
 3   AMZN.O  2138 non-null   float64
 4   GS.N    2138 non-null   float64
 5   SPY     2138 non-null   float64
 6   .SPX    2138 non-null   float64
 7   .VIX    2138 non-null   float64
 8   EUR=    2216 non-null   float64
 9   XAU=    2211 non-null   float64
 10  GDX     2138 non-null   float64
 11  GLD     2138 non-null   float64
dtypes: float64(12)
memory usage: 225.1 KB


In [155]:
selected = ['AAPL.O', 'MSFT.O', 'INTC.O', 'AMZN.O', 'GS.N']

In [156]:
data = pd.DataFrame(raw[selected].dropna())
data

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N
Date,,,,,
2010-01-04,30.572827,30.950,20.88,133.90,173.08
2010-01-05,30.625684,30.960,20.87,134.69,176.14
2010-01-06,30.138541,30.770,20.80,132.25,174.26
2010-01-07,30.082827,30.452,20.60,130.00,177.67
2010-01-08,30.282827,30.660,20.83,133.52,174.31
...,...,...,...,...,...
2018-06-25,182.170000,98.390,50.71,1663.15,221.54
2018-06-26,184.430000,99.080,49.67,1691.09,221.58
2018-06-27,184.160000,97.540,48.76,1660.51,220.18


In [157]:
for stock in selected:
    data['Returns_' + stock] = np.log(data[stock] / data[stock].shift(1)).dropna()
    data['Returns_' + stock].fillna(0, inplace=True)
data

/var/folders/2b/tgv3r4pj32b_81f22_mtv7bh0000gn/T/ipykernel_12595/2098642776.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Returns_' + stock].fillna(0, inplace=True)
/var/folders/2b/tgv3r4pj32b_81f22_mtv7bh0000gn/T/ipykernel_12595/2098642776.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alwa

,AAPL.O,MSFT.O,INTC.O,AMZN.O,GS.N,Returns_AAPL.O,Returns_MSFT.O,Returns_INTC.O,Returns_AMZN.O,Returns_GS.N
Date,,,,,,,,,,
2010-01-04,30.572827,30.950,20.88,133.90,173.08,0.000000,0.000000,0.000000,0.000000,0.000000
2010-01-05,30.625684,30.960,20.87,134.69,176.14,0.001727,0.000323,-0.000479,0.005883,0.017525
2010-01-06,30.138541,30.770,20.80,132.25,174.26,-0.016034,-0.006156,-0.003360,-0.018282,-0.010731
2010-01-07,30.082827,30.452,20.60,130.00,177.67,-0.001850,-0.010389,-0.009662,-0.017160,0.019379
2010-01-08,30.282827,30.660,20.83,133.52,174.31,0.006626,0.006807,0.011103,0.026717,-0.019093
...,...,...,...,...,...,...,...,...,...,...
2018-06-25,182.170000,98.390,50.71,1663.15,221.54,-0.014983,-0.020323,-0.034690,-0.031090,-0.020020
2018-06-26,184.430000,99.080,49.67,1691.09,221.58,0.012330,0.006988,-0.020722,0.016660,0.000181
2018-06-27,184.160000,97.540,48.76,1660.51,220.18,-0.001465,-0.015665,-0.018491,-0.018249,-0.006338


In [158]:
split = int(len(data)*0.8)
data_train = data.iloc[:split].copy()
data_test = data.iloc[split:].copy()

In [159]:
prices_train = data_train['AAPL.O'].values
prices_train

array([ 30.57282657,  30.62568366,  30.13854129, ..., 116.98      ,
       117.63      , 117.55      ])

In [160]:
Volatility = []
MA = []
Return = []
ma_sum = 0

In [161]:
for i in range(10):
    Volatility.append(0)
    MA.append(0)
    Return.append(0)
    ma_sum += prices_train[i]

for i in range(0, len(prices_train)):
    if i >= 10:
        tail_close = prices_train[i-10]
        prev_close = prices_train[i-1]
        head_close = prices_train[i]
        ma_sum = (ma_sum - tail_close + head_close)
        ma_curr = ma_sum / 10
        MA.append(ma_curr)
        Return.append(((head_close - prev_close) / prev_close) * 100)
        vol_sum = 0
        for j in range(0, 10):
            curr_vol = abs(ma_curr - prices_train[i-j])
            vol_sum += (curr_vol ** 2)
        Volatility.append(vol_sum / 10)

In [162]:
prices_train = prices_train[10:]
Volatility = Volatility[10:]
Return = Return[10:]

In [163]:
model = GaussianHMM(n_components=3, covariance_type="full", n_iter=1000)

In [164]:
obs = []
for i in range(0, len(Volatility)):
    arr = []
    arr.append(Volatility[i])
    arr.append(Return[i])
    obs.append(arr)

In [165]:
model.fit(obs)

GaussianHMM(covariance_type='full', n_components=3, n_iter=1000)

In [166]:
predictions = model.predict(obs)

In [167]:
regime_vol = {}
regime_ret = {}

for i in range(3):
    regime_vol[i] = []
    regime_ret[i] = []

for i in range(len(predictions)):
    regime_vol[predictions[i]].append(Volatility[i])
    regime_ret[predictions[i]].append(Return[i])

In [168]:
class Distribution(object):

    def __init__(self, dist_names_list = []):
        self.dist_names = ['norm','lognorm','expon', 'gamma',
                           'beta', 'rayleigh', 'norm', 'pareto']
        self.dist_results = []
        self.params = {}

        self.DistributionName = ""
        self.PValue = 0
        self.Param = None
        self.isFitted = False


    def Fit(self, y):
        self.dist_results = []
        self.params = {}
        for dist_name in self.dist_names:
            dist = getattr(scipy.stats, dist_name)
            param = dist.fit(y)
            self.params[dist_name] = param
            #Applying the Kolmogorov-Smirnov test
            D, p = scipy.stats.kstest(y, dist_name, args=param);
            self.dist_results.append((dist_name,p))

        #select the best fitted distribution
        sel_dist,p = (max(self.dist_results,key=lambda item:item[1]))
        #store the name of the best fit and its p value
        self.DistributionName = sel_dist
        self.PValue = p
        self.isFitted = True

        return self.DistributionName, self.PValue

    def PDF(self, x):
        dist = getattr(scipy.stats, self.DistributionName)
        n = dist.pdf(x, *self.params[self.DistributionName])
        return n

In [169]:
vols = []
rets = []
today_regime = predictions[-1]
for i in range(3):
    vol_dist = Distribution()
    vol_dist.Fit(regime_vol[i])
    vols.append(vol_dist.PDF(Volatility[-1]))
    ret_dist = Distribution()
    ret_dist.Fit(regime_ret[i])
    rets.append(ret_dist.PDF(Return[-1]))

/Users/jingyizhang/Library/Python/3.9/lib/python/site-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/Users/jingyizhang/Library/Python/3.9/lib/python/site-packages/scipy/stats/_continuous_distns.py:709: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)


In [170]:
bear = -1
bull = -1
neg_return = 1
pos_return = -1
low_vol = 100
for i in range(3):
    if sum(regime_ret[i]) / len(regime_ret[i]) < neg_return:
        neg_return = sum(regime_ret[i]) / len(regime_ret[i])
        bear = i
    if sum(regime_ret[i]) / len(regime_ret[i]) > pos_return:
        pos_return = sum(regime_ret[i]) / len(regime_ret[i])
        bull = i

In [178]:
model.means_

array([[6.41107550e-01, 1.26096134e-01],
       [9.17139097e+00, 7.91969444e-03],
       [2.63879962e+00, 1.01470653e-01]])

In [171]:
bear, bull

(1, 0)

In [172]:
unique_elements, counts = np.unique(predictions, return_counts=True)
value_counts = dict(zip(unique_elements, counts))

print(value_counts)


{0: 699, 1: 282, 2: 719}


In [173]:
for i in range(len(predictions)):
    if predictions[i] == bear:
        predictions[i] = -1
    elif predictions[i] == bull:
        predictions[i] = 1
    else:
        if i > 0:
            predictions[i] = predictions[i-1]

In [174]:
unique_elements, counts = np.unique(predictions, return_counts=True)
value_counts = dict(zip(unique_elements, counts))

print(value_counts)

{-1: 616, 1: 1084}


In [175]:
returns = data_train['Returns_AAPL.O'][10:] * predictions

In [176]:
returns_df = pd.DataFrame(returns)
returns_sum = returns_df.sum().apply(np.exp)
print(returns_sum)

Returns_AAPL.O    3.573636
dtype: float64


In [177]:
pd.DataFrame(data_train['Returns_AAPL.O'][10:]).sum().apply(np.exp)

Returns_AAPL.O    3.995779
dtype: float64

In [179]:
rets_train = np.concatenate([data_train['Returns_' + stock].values.reshape(-1,1) for stock in selected])

In [188]:
def train_hmm(data, n_iter=50):
    best_model = None
    best_score = float('-inf')
    
    for _ in range(n_iter):
        model = GaussianHMM(n_components=3, covariance_type="full", n_iter=100)
        model.fit(data)
        score = model.score(data)
        if score > best_score:
            best_score = score
            best_model = model
    
    return best_model

In [190]:
model = train_hmm(rets_train)

In [192]:
model.means_

array([[ 0.00114044],
       [-0.0001997 ],
       [-0.00116171]])